In [1]:
DATA_NAME = 'spaceNet-full' 
TRANSFORM = 'learned'
CHANNEL = ''

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_learned_full_spaceNet.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,0,2,3,4,5,6,7,8,9,10,...,54,55,56,57,58,59,60,61,62,63
0,57.21,0.00,-0.00,-0.00,0.00,0.00,-0.00,-0.00,0.00,-0.00,...,-0.00,-0.00,-0.00,-0.00,0.00,-0.00,0.00,0.00,0.00,0.00
2,0.00,82.09,-0.00,-0.01,-0.00,-0.00,0.00,0.00,0.00,-0.00,...,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0.00,-0.00,-0.00,0.00
3,-0.00,-0.00,22.61,-0.00,-0.00,0.00,0.00,0.00,0.00,0.00,...,-0.00,0.00,0.00,0.00,-0.00,0.00,0.00,0.00,-0.00,0.00
4,-0.00,-0.01,-0.00,94.09,0.00,0.00,-0.00,0.00,-0.00,-0.00,...,0.00,0.00,0.00,0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00
5,0.00,-0.00,-0.00,0.00,57.49,-0.00,-0.00,0.00,-0.00,-0.00,...,0.00,0.00,0.00,-0.00,-0.00,-0.00,0.00,-0.00,0.00,0.00
6,0.00,-0.00,0.00,0.00,-0.00,43.71,0.00,-0.00,0.00,0.00,...,-0.00,0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,-0.00,0.00
7,-0.00,0.00,0.00,-0.00,-0.00,0.00,24.62,0.00,0.00,0.00,...,-0.00,0.00,-0.00,-0.00,0.00,-0.00,0.00,0.00,0.00,-0.00
8,-0.00,0.00,0.00,0.00,0.00,-0.00,0.00,1.92,0.00,-0.00,...,-0.00,0.00,-0.00,-0.00,-0.00,-0.00,0.00,0.00,0.00,0.00
9,0.00,0.00,0.00,-0.00,-0.00,0.00,0.00,0.00,9.37,0.00,...,-0.00,0.00,0.00,-0.00,-0.00,-0.00,-0.00,0.00,-0.00,-0.00
10,-0.00,-0.00,0.00,-0.00,-0.00,0.00,0.00,-0.00,0.00,35.87,...,0.00,-0.00,0.00,-0.00,0.00,0.00,-0.00,0.00,-0.00,0.00


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,0,2,3,4,5,6,7,8,9,10,...,54,55,56,57,58,59,60,61,62,63
0,1.00000,0.00002,-0.00000,-0.00003,0.00002,0.00006,-0.00002,-0.00008,0.00002,-0.00002,...,-0.00001,-0.00004,-0.00006,-0.00001,0.00001,-0.00001,0.00001,0.00000,0.00004,0.00004
2,0.00002,1.00000,-0.00002,-0.00006,-0.00002,-0.00006,0.00001,0.00003,0.00000,-0.00004,...,-0.00000,-0.00000,-0.00004,-0.00005,-0.00000,-0.00004,0.00001,-0.00001,-0.00003,0.00003
3,-0.00000,-0.00002,1.00000,-0.00001,-0.00000,0.00003,0.00002,0.00007,0.00002,0.00004,...,-0.00003,0.00005,0.00000,0.00002,-0.00002,0.00002,0.00004,0.00004,-0.00001,0.00003
4,-0.00003,-0.00006,-0.00001,1.00000,0.00003,0.00002,-0.00002,0.00006,-0.00001,-0.00001,...,0.00006,0.00001,0.00002,0.00003,0.00001,-0.00003,0.00001,-0.00003,0.00003,-0.00005
5,0.00002,-0.00002,-0.00000,0.00003,1.00000,-0.00005,-0.00001,0.00002,-0.00002,-0.00003,...,0.00003,0.00002,0.00001,-0.00001,-0.00001,-0.00004,0.00000,-0.00001,0.00005,0.00003
6,0.00006,-0.00006,0.00003,0.00002,-0.00005,1.00000,0.00000,-0.00001,0.00003,0.00001,...,-0.00001,0.00002,0.00001,-0.00001,0.00006,-0.00001,0.00001,-0.00002,-0.00001,0.00006
7,-0.00002,0.00001,0.00002,-0.00002,-0.00001,0.00000,1.00000,0.00001,0.00001,0.00000,...,-0.00006,0.00003,-0.00002,-0.00002,0.00001,-0.00002,0.00006,0.00001,0.00001,-0.00003
8,-0.00008,0.00003,0.00007,0.00006,0.00002,-0.00001,0.00001,1.00000,0.00002,-0.00001,...,-0.00002,0.00005,-0.00001,-0.00004,-0.00002,-0.00002,0.00001,0.00006,0.00001,0.00000
9,0.00002,0.00000,0.00002,-0.00001,-0.00002,0.00003,0.00001,0.00002,1.00000,0.00000,...,-0.00000,0.00004,0.00003,-0.00006,-0.00002,-0.00005,-0.00006,0.00000,-0.00006,-0.00002
10,-0.00002,-0.00004,0.00004,-0.00001,-0.00003,0.00001,0.00000,-0.00001,0.00000,1.00000,...,0.00000,-0.00001,0.00003,-0.00001,0.00003,0.00003,-0.00001,0.00001,-0.00000,0.00001


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.06656612230083869

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[1.58957910e+02 1.21523693e+02 1.15883174e+02 1.13066345e+02
 1.04807137e+02 9.16705036e+01 7.76428785e+01 6.81236864e+01
 6.68453966e+01 6.48611975e+01 6.30630448e+01 6.01236701e+01
 5.97205092e+01 5.63885646e+01 4.85908904e+01 4.34941526e+01
 4.26815856e+01 4.21073995e+01 4.03149796e+01 3.72354276e+01
 3.44573831e+01 3.35008959e+01 3.26578150e+01 3.13751571e+01
 2.96228261e+01 2.81164995e+01 2.69877608e+01 2.61554953e+01
 2.44011753e+01 2.20272267e+01 1.83267745e+01 1.75385274e+01
 1.61878093e+01 1.58098108e+01 1.54772682e+01 1.50791230e+01
 1.34886808e+01 1.20511902e+01 1.13996014e+01 1.03337486e+01
 9.16313743e+00 8.63916688e+00 8.03161494e+00 7.67489568e+00
 7.31424460e+00 6.98699356e+00 6.21574030e+00 5.32536914e+00
 4.75132856e+00 2.71738884e+00 1.74074547e+00 1.53553399e+00
 9.84445527e-01 6.01527398e-01 1.71205722e-01 1.85603437e-30]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8,9,...,46,47,48,49,50,51,52,53,54,55
0,-0.012320,-0.048451,-0.003225,0.995143,-0.012326,-0.007466,-0.003557,-0.000248,-0.001271,-0.005642,...,-0.000860,-0.004836,-0.017333,-0.009004,-0.003148,-0.039687,-0.000704,-0.002873,-0.014105,-0.002219
1,-0.010880,0.967185,-0.002404,0.028676,-0.011157,-0.006072,-0.002621,-0.000181,-0.000922,-0.004389,...,-0.000637,-0.003679,-0.018295,-0.007538,-0.002335,-0.171453,-0.000510,-0.002123,-0.013471,-0.001595
2,-0.000483,0.007183,-0.000098,0.001075,-0.000696,-0.000279,-0.000079,-0.000004,-0.000052,-0.000147,...,-0.000019,-0.000176,-0.000835,-0.000267,-0.000123,0.718152,-0.000027,-0.000113,-0.000452,-0.000035
3,-0.024880,0.202595,-0.005057,0.046007,-0.025210,-0.013046,-0.005661,-0.000403,-0.001973,-0.009313,...,-0.001370,-0.007824,-0.044233,-0.016581,-0.004972,0.643343,-0.001084,-0.004438,-0.031091,-0.003405
4,-0.028784,0.105791,-0.005424,0.038163,-0.029295,-0.014211,-0.006007,-0.000416,-0.002069,-0.010112,...,-0.001425,-0.008459,-0.057208,-0.018252,-0.005319,0.165134,-0.001149,-0.004751,-0.037381,-0.003650
5,-0.033141,0.049629,-0.005080,0.025298,-0.034012,-0.014215,-0.005721,-0.000397,-0.001935,-0.009727,...,-0.001355,-0.008154,-0.093176,-0.019062,-0.005027,0.061290,-0.001095,-0.004512,-0.046813,-0.003442
6,-0.042870,0.024747,-0.004437,0.015112,-0.043965,-0.013673,-0.004991,-0.000336,-0.001641,-0.008846,...,-0.001149,-0.007244,0.980029,-0.019398,-0.004351,0.027946,-0.000927,-0.003917,-0.080052,-0.002932
7,-0.030499,0.006578,-0.001740,0.004707,-0.032934,-0.005907,-0.001915,-0.000126,-0.000617,-0.003588,...,-0.000439,-0.002936,0.028377,-0.008926,-0.001692,0.007669,-0.000353,-0.001442,-0.425108,-0.001165
8,-0.067105,0.012766,-0.003394,0.008636,-0.072813,-0.011843,-0.003788,-0.000247,-0.001258,-0.007163,...,-0.000834,-0.005716,0.048016,-0.018323,-0.003336,0.013984,-0.000673,-0.002983,0.841080,-0.002187
9,-0.073248,0.009869,-0.002852,0.006768,-0.082191,-0.009989,-0.003189,-0.000202,-0.001033,-0.005970,...,-0.000736,-0.004754,0.033326,-0.016030,-0.002731,0.010710,-0.000561,-0.002427,0.161200,-0.001815


In [13]:
cos_dist = spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine')
cos_dist = np.sort(np.concatenate(cos_dist))[:len(GROUPS)]
cos_dist


array([0.00485747, 0.01997132, 0.02232754, 0.03281526, 0.04549374,
       0.05203407, 0.10007937, 0.10338124, 0.10533655, 0.11725176,
       0.13008478, 0.14143686, 0.14158142, 0.14200177, 0.15892043,
       0.16195212, 0.17177627, 0.1853875 , 0.18987459, 0.19284598,
       0.19781407, 0.19830294, 0.20402294, 0.20494197, 0.20799622,
       0.20912432, 0.21231001, 0.21634997, 0.21728007, 0.23300752,
       0.23808187, 0.24691395, 0.24699468, 0.25133529, 0.26133993,
       0.26945249, 0.27101711, 0.27186556, 0.28184833, 0.28473001,
       0.29073169, 0.29596386, 0.2989265 , 0.30498036, 0.32155129,
       0.32745566, 0.33100749, 0.34074239, 0.34770504, 0.3566568 ,
       0.35717815, 0.38828896, 0.414868  , 0.45492753, 0.46760564,
       0.49313564])